5 6 2 2 1 1 3 2

In [29]:
w, h, f, c, x1, y1, x2, y2 = map(int, input().split())

after_x = max(w - f, f)
after_y = h // (c + 1)
mul = c + 1

total = 0
total += mul * max(0, x2 - max(x1, min(f, w - f))) * (y2 - y1)
total += 2 * mul * (y2 - y1) * max(0, min(x2, min(f, w - f)) - x1)
print(w * h - total)

 4 8 3 0 1 1 3 2


30


In [22]:
2 * mul * (y2 - y1) * max(0, min(x2, min(f, w - f)) - x1)

6

In [4]:
mul * (y2 - y1) * max(0, max(after_x - f, 2 * f - w) - x1)

0

In [2]:
print(after_x, after_y)

3 2


## 8-2. 데이터 전처리

- 데이터 추출
    - df1.head() : 상위 n개 데이터 추출
    - df1.tail() : 하위 n개 데이터 추출
    - df1.iloc[n:m] : n번째 행 부터 m번째 행 까지 추출
    - df1['col'] : 특정 col을 추출 (Series 형태로 추출)
    - df1[['col1', 'col2']] : 한 개 이상의 col을 데이터 프레임형태로 추출

- 데이터 정렬
    - df1.sort_values(by = col, ascending = True) : 특정 col열을 오름차순/ 내림차순으로 정렬

- 데이터 필터 (층별화 - 특정 범주항목 별, 데이터를 나누어 분석)
    - df1.loc[cond1] : cond1 (특정 조건을 만족하는, 비교구문에 있어 True 출력되는 행) 에 대해 데이터를 출력
    - df1.loc[cond1, new_col] = 'new_data' : cond1에 대해 new_data를 new_col으로 선언 (특정 조건에 대해 새로운 데이터를 파생변수로 선언)

- 데이터 요약
    - df1. pivot_table(index = 구분하고자 하는 항목, values = 계산하고자 하는 항목, aggfunc = 통계량)
    - 특정 항목을 기준으로 특정 값을 원하는 통계량에 대해 계산
    - columns = : 구분하고자 하는 항목을 column으로 생성
    - margins = 특정 통계량을 행/열 단위로 계산
    - fill_values : 요약을 수행할 때 발생한 결측값을 특정 값으로 대치

- 파생면수 생성
    - df1.apply(function) : 특정 column의 각 row를 사용자가 만든(정의한) function에 넣어 계산
 
- 날짜 데이터 처리
    - pd.to_datetime(df1[col]) : 특정 날짜 형식(col)의 자료를 날짜형태 타입으로 변환
    - df1[col_dt].dt.year : 연도 추출
    - df1[col_dt].dt.month : 월 추출
    - df1[col_dt].dt.day : 일자 추출
    - df1[col_dt].dt.day_name() : 요일 추출
    - df1[col_dt].dt.isocalender().week : 주차 추출 (work week)

- 결측값 처리
- 데이터 재구조화 및 병합

**결측값 처리**

- 결측값 (missing value -> Nan, Not A Number)
- 결측값 제거 : 결측값이 존재하는 행을 제거
- 결측값 대치 : 결측값을 다른 값으로 채워 넣는 방법
    - 단순대치 : 평균이나 중강값과 같은 해당col을 대표할 수 있는 값으로 대치
    - 알고리즘 대치 : 머신러닝 알고리즘들을 활용해 해당 결측값과 유사한 값으로 대치
    - 보간법 : 특정 시간 간격이나 서열 간격에 맞춰 데이터를 생성해 대치

In [5]:
import pandas as pd

In [7]:
df1 = pd.read_csv('data/01_Data.csv') # 데이터 호출
print(df1.shape) # 데이터의 행, 열 개수 확인
df1.head() # 데이터 확인

(51301, 20)


,Unnamed: 0,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,Customer_Type,Age,Address1,State,Overdue_count,Overdue_Type,Gender,Bank,Credit_Score
0,0,1,66758234,렌탈,일반계약,영업방판,2022-05-09,60,CMS,SLX-1,96900,개인,42.0,경기도,계약확정,0,없음,여자,새마을금고,496.0
1,1,2,66755948,렌탈,교체계약,영업방판,2023-02-23,60,카드이체,SLX-1,102900,개인,39.0,경기도,계약확정,0,없음,남자,현대카드,876.0
2,2,3,66756657,렌탈,일반계약,홈쇼핑/방송,2022-03-03,60,CMS,SLX-1,96900,개인,48.0,경기도,계약확정,0,없음,여자,우리은행,512.0
3,3,4,66423450,멤버십,멤버십3유형,재계약,2022-05-16,12,CMS,SLX-1,66900,개인,39.0,경기도,계약확정,0,없음,남자,농협회원조합,777.0
4,4,5,66423204,멤버십,멤버십3유형,재계약,2022-05-13,12,CMS,SLX-1,66900,개인,60.0,경기도,기간만료,12,있음,남자,농협회원조합,561.0


In [11]:
# 결측값 확인
df1.info() # Non-Null count : 결측값이 아닌 데이터 수

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51301 entries, 0 to 51300
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     51301 non-null  int64  
 1   Index          51301 non-null  int64  
 2   Member_ID      51301 non-null  int64  
 3   Sales_Type     51301 non-null  object 
 4   Contract_Type  51301 non-null  object 
 5   Channel        51301 non-null  object 
 6   Datetime       51301 non-null  object 
 7   Term           51301 non-null  int64  
 8   Payment_Type   51301 non-null  object 
 9   Product_Type   51301 non-null  object 
 10  Amount_Month   51301 non-null  int64  
 11  Customer_Type  51299 non-null  object 
 12  Age            44329 non-null  float64
 13  Address1       51299 non-null  object 
 14  State          51301 non-null  object 
 15  Overdue_count  51301 non-null  int64  
 16  Overdue_Type   51301 non-null  object 
 17  Gender         51301 non-null  object 
 18  Bank  

In [13]:
df1.isnull().sum() # 특정 데이터가 결측값인지 아닌지 출력 -> sum함수를 사용하면 True값의 개수를 column별 합을 계산

Unnamed: 0          0
Index               0
Member_ID           0
Sales_Type          0
Contract_Type       0
Channel             0
Datetime            0
Term                0
Payment_Type        0
Product_Type        0
Amount_Month        0
Customer_Type       2
Age              6972
Address1            2
State               0
Overdue_count       0
Overdue_Type        0
Gender              0
Bank             2759
Credit_Score     8841
dtype: int64

In [16]:
# 결측값을 제거 (특정 행에 결측값이 존재하면, 해당 행의 정보를 삭제)
df1_clean = df1.dropna()
df1_clean.shape

(40597, 20)

In [17]:
# 결측값이 많은 열을 제거한 뒤 결측값을 제거하여 분석
# drop(): 특정 행 또는 열을 삭제
df1_clean2 = df1.drop(columns = ['Age', 'Bank', 'Credit_Score']).dropna()
df1_clean2.shape

(51299, 17)

In [18]:
# 결측값 대치 (단순대치)
df1['Credit_Score'].tail(10)

51291      NaN
51292      NaN
51293    978.0
51294      NaN
51295    911.0
51296      NaN
51297    574.0
51298    972.0
51299    888.0
51300    538.0
Name: Credit_Score, dtype: float64

In [20]:
# 결측값을 0으로 대치하여 처리
df1['Credit_Score_clean'] = df1['Credit_Score'].fillna(0)
df1['Credit_Score_clean'].tail(10)

51291      0.0
51292      0.0
51293    978.0
51294      0.0
51295    911.0
51296      0.0
51297    574.0
51298    972.0
51299    888.0
51300    538.0
Name: Credit_Score_clean, dtype: float64

원시자료 설명서(DB설계서) -> 결측값이 다른 숫자값으로 대치되었는지 확인할 수 있다.

In [31]:
# 결측값을 평균으로 대치하여 처리
df1['Credit_Score_clean2'] = df1['Credit_Score'].fillna(df1['Credit_Score'].mean())
df1['Credit_Score_clean2'].tail(10)

51291    826.838342
51292    826.838342
51293    978.000000
51294    826.838342
51295    911.000000
51296    826.838342
51297    574.000000
51298    972.000000
51299    888.000000
51300    538.000000
Name: Credit_Score_clean2, dtype: float64

## 8-3. 기술통계량 확인

- 데이터의 특성을 알기 위해 기본적으로 확인하는 통계량
- 연속형 :
    - 대표값 (평균, 중앙값): 해당 연속형 자료를 대표하는 수치
        - 평균의 경우, 이상치(outlier)에 의해 매우 높거나 낮게 계산될 수 있으므로, 반드시 중위수(중앙값)와 같이 확인
        - A : 1, 3, 2, 5, 4 -> 평균 : 3
        - A : 1, 2, 3, 4, 5 -> 중앙값 : 3
        - B : 1, 3, 2, 4, 10000 -> 평균 : 2002
        - B : 1, 2, 3, 4, 10000 -> 중앙값 : 3
 
    - 산포 (표준편차) : 해당 연속형자료의 데이터들이 대표값으로부터 얼마나 떨어져 있는지 나타내는 통계량
        - 산포는 데이터가 얼마나 대표값에 대해 정확한가를 나타내는 척도
        - 데이터가 많이 수집 될 수록, 대표값에 대해 모여있는 분포를 나타낸(중심극한의 정리)
        - 데이터가 많아지면, 산포가 줄어들 가능성이 있음

    - 분포의 모양 : 해당 연속형 자료의 분포의 모양, 시각화
        - 정규분포, 정규분포가 아닌 데이터
        - 분포의 모양에 따라 가설검정 및 통계적 분석 방법이 달라질 수 있음
        - 데이터 수집 관점

- 범주형 : 항목과 빈도수 확인 (Unique / Frequency), 비율